In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
images = np.load("subset_1.npy")

In [4]:
images.shape

(400, 101250)

In [5]:
one_image = np.reshape(images[2, :], (150, 225, 3))

In [ ]:
plt.imshow(one_image, cmap='grey')

In [ ]:
fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(7.5, 5))

for i in range(5):
    for j in range(5):
        axs[i, j].imshow(np.reshape(images[(i*10)+j, :], (25, 50)), cmap='grey')
        axs[i, j].set_xticks([])
        axs[i, j].set_yticks([])

In [ ]:
images = images / 255

In [ ]:
labels = np.load("labels.npy")

In [ ]:
images.shape

In [ ]:
labels.shape

In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
labels = OneHotEncoder(sparse_output=False).fit_transform(labels.reshape(-1, 1))

In [ ]:
labels

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(images, labels, test_size=0.30)

In [ ]:
type(train_X)

In [ ]:
train_X = torch.from_numpy(train_X).to(torch.float)
train_y = torch.from_numpy(train_y).to(torch.float)
test_X = torch.from_numpy(test_X).to(torch.float)
test_y = torch.from_numpy(test_y).to(torch.float)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_layers = [
            nn.Linear(101250, 200),
            nn.Linear(200, 10)
        ]

        self.output_layer = nn.Linear(10, 4)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        return F.softmax(F.sigmoid(self.output_layer(x)))

In [ ]:
my_network = Net()

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(my_network.parameters(), learning_rate=0.01)

In [ ]:
my_network(train_X[:5])

In [ ]:
batch_size = 50
for epoch in range(5):
    loss = 0.0
    for idx in range(0, train_X.shape[0], batch_size):
        optimizer.zero_grad()
        outputs = my_network(train_X[idx:idx+batch_size])
        new_loss = criterion(outputs, train_y[idx:idx+batch_size])
        new_loss.backward()
        optimizer.step()
        loss = loss + new_loss.item()
    print(f"Epoch {epoch} loss: {loss:.3f}")
    
    test_classes = my_network(test_X)
    test_classes = test_classes.argmax(axis=1)
    print(f"Test Accuracy: {accuracy_score(test_classes, test_y.argmax(axis=1))}")